### Sentiment Analysis

#### 1. Connect to the server

In [15]:
import pandas as pd
import numpy as np
from clickhouse_driver import Client

In [16]:
# macro database information, do not change
host_name = 'chenlin01.fbe.hku.hk'
user_name = 'mfin7037_best_students'
pswd = 'alanisthecoolest123'
db_name = 'tiingo'
port = 9000

In [17]:
client = Client(
    host=host_name,
    user=user_name,
    password=pswd,
    database=db_name,
    port=port,
)

In [20]:
client.execute('SHOW DATABASES')

[('default',), ('tiingo',)]

In [22]:
client.execute('SHOW TABLES')

[('news',)]

In [27]:
client.execute('USE tiingo')
client.execute('SELECT * FROM news limit 2')

[('2018-05-02T12:14:50.841934+00:00',
  'Yamana Gold Inc. (NYSE:AUY) shares are down more than -8.65% this year and recently decreased -0.70% or -$0.02 to settle at $2.85. SM Energy Company (NYSE:SM), on the other hand, is up 6.61% year to date as of 05/01/2018. It currently trades at $23.54 and has returned 3.11% during the past week. Yamana Gold Inc.…',
  10471811.0,
  '2018-05-02T11:06:33+00:00',
  'stocknewsgazette.com',
  'Energy/Materials/Stock',
  'sm',
  'auy/sm',
  'Yamana Gold Inc. (AUY) vs. SM Energy Company (SM): Which is the Better Investment?',
  'stocknewsgazette.com',
  'https://stocknewsgazette.com/2018/05/02/yamana-gold-inc-auy-vs-sm-energy-company-sm-which-is-the-better-investment/'),
 ('2018-05-02T12:15:20.345795+00:00',
  'AGNC Investment Corp. (NASDAQ:AGNC) shares are down more than -6.59% this year and recently decreased -0.32% or -$0.06 to settle at $18.86. B2Gold Corp. (NYSE:BTG), on the other hand, is down -9.03% year to date as of 05/01/2018. It currently tra